In [1]:
from typing import Optional, List, Tuple
from tqdm.notebook import tqdm
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline, Conversation, LlamaForCausalLM, LlamaTokenizer
from langchain import HuggingFaceHub
from langchain.llms import HuggingFacePipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

CHECK HuggingFace spaces

# Usage of Tiny Llama chat hugging face

In [2]:
torch.cuda.empty_cache()

In [3]:
MODEL_PATH = 'local_model/tiny_llama'
TOKENIZER_PATH = 'local_tokenizer/tiny_llama'
MAX_ANSWER_LEN = 128
MIN_ANSWER_LEN = 12
RETURN_SENTENCE = 1
BATCH_SIZE = 1
torch.manual_seed(21)
MODEL_TOKENIZER = LlamaTokenizer.from_pretrained(TOKENIZER_PATH)
MODEL = LlamaForCausalLM.from_pretrained(MODEL_PATH,
                                        device_map='auto',
                                        torch_dtype=torch.float16,
                                        low_cpu_mem_usage=True,
                                        )
#MODEL_TOKENIZER.apply_chat_template(chat, tokenize=False) 'cuda' if torch.cuda.is_available() else
DEVICE =  'cpu'


In [4]:
chat_bot = pipeline(
    'conversational',
    model=MODEL,
    tokenizer=MODEL_TOKENIZER,
    max_length=MAX_ANSWER_LEN,
    min_length_for_response=MIN_ANSWER_LEN,
    num_return_sequences=RETURN_SENTENCE, 
    batch_size=BATCH_SIZE,
)

# Dataset Builders

In [5]:
class message_dataset(Dataset):
    def __init__(self, path:str):
        self.messages = pd.read_csv(path, names=['message'])

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = self.messages.message.iloc[idx]
        return message    

In [6]:
negative_message_dataset = message_dataset('messages/negative_mess.csv')

In [7]:
negative_message_loader = DataLoader(negative_message_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
def inference(pipeline, loader, prompt:str=None, path:str='replies/tmp.csv'):
    df = pd.DataFrame({'reply':[]})
    with torch.no_grad():
        for message in tqdm(loader):
            conv = Conversation(message)
            if prompt:
                conv.add_message({'role':'system', 'content':f'{prompt}'})
                

            reply = pipeline(conv)
            print(reply.messages[-1]["content"])
            df.loc[len(df.index)] = [reply.messages[-1]["content"]] 
    df.to_csv(path)
    return df

# experement 1

In [9]:
prompt = 'Answer user complains'

In [10]:
replies = inference(chat_bot, negative_message_loader, prompt)

  0%|          | 0/48 [00:00<?, ?it/s]

d:\anaconda\Lib\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Sure, I'd be happy to help! Here's a possible solution:

1. Identify the problem: The first step in resolving a customer complaint is to identify the problem. In this case, the customer is complaining about the quality of the product.

2. Collect relevant information: Once you've identified the problem, gather as much relevant information as possible. This could include the customer's name, order number, product description, and any relevant details about the complaint.


Sure, I'd be happy to help! Here's a possible solution:

1. Identify the problem: The first step in resolving a customer complaint is to identify the problem. In this case, the customer is complaining about the quality of the product.

2. Collect relevant information: Once you've identified the problem, gather as much relevant information as possible. This could include the customer's name, order number, product description, and any relevant details about the complaint.


Sure, I'd be happy to help! Here's a possible 

--- Logging error ---
Traceback (most recent call last):
  File "d:\anaconda\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "d:\anaconda\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "d:\anaconda\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "d:\

Sure, I'd be happy to help! Here's a possible solution:

1. Identify the problem: The first step in resolving a customer complaint is to identify the problem. In this case, the customer is complaining about the quality of the product.

2. Collect relevant information: Once you've identified the problem, gather as much relevant information as possible. This could include the customer's name, order number, product description, and any relevant details about the complaint.


Sure, I'd be happy to help! Here's a possible solution:

1. Identify the problem: The first step in resolving a customer complaint is to identify the problem. In this case, the customer is complaining about the quality of the product.

2. Collect relevant information: Once you've identified the problem, gather as much relevant information as possible. This could include the customer's name, order number, product description, and any relevant details about the complaint.


Sure, I'd be happy to help! Here's a possible 

In [19]:
res = pd.concat([negative_message_dataset.messages, replies], axis=1)
res.to_csv('joined/You_are_working_at_customer_chat_support_delivery_service.csv')

# experiment 2

In [20]:
path = 'replies/short_work.csv'

In [21]:
prompt = 'You are chat support for delivery service, answer shortly to customer'

In [22]:
replies = inference(chat_bot, negative_message_loader, prompt, path)

  0%|          | 0/48 [00:00<?, ?it/s]

Dear [Customer Name],

Thank you for reaching out to us regarding your delivery service. We understand how important it is for you to receive your order on time, and we are committed to providing the best possible service.

We have received your order and are currently processing it. Once it has been confirmed, we will send you an email with the estimated delivery date. We will also provide you with a tracking number so that you can track your package's progress.

We take
Dear [Customer Name],

Thank you for reaching out to us regarding your delivery service. We understand how important it is for you to receive your order on time, and we are committed to providing the best possible service.

We have received your order and are currently processing it. Once it has been confirmed, we will send you an email with the estimated delivery date. We will also provide you with a tracking number so that you can track your package's progress.

We take
Dear [Customer Name],

Thank you for reaching 

In [23]:
res = pd.concat([negative_message_dataset.messages, replies], axis=1)
res.to_csv('joined/short_work.csv')

# experiment 3

In [24]:
path = 'replies/handle_problem.csv'
prompt = 'Handle users problem as support'
replies = inference(chat_bot, negative_message_loader, prompt, path)
res = pd.concat([negative_message_dataset.messages, replies], axis=1)
res.to_csv('joined/handle_problem.csv')

  0%|          | 0/48 [00:00<?, ?it/s]

I'm not able to interact with users directly, but I can provide you with a general approach for handling user problems in a support system. Here are some steps you can follow:

1. Identify the problem: the first step in handling a user problem is to identify the problem. This involves asking the user for as much information as possible, such as the specific issue they are experiencing, the steps they took to reproduce the problem, and any relevant context or background information.

2. Assign a support ticket
I'm not able to interact with users directly, but I can provide you with a general approach for handling user problems in a support system. Here are some steps you can follow:

1. Identify the problem: the first step in handling a user problem is to identify the problem. This involves asking the user for as much information as possible, such as the specific issue they are experiencing, the steps they took to reproduce the problem, and any relevant context or background information

# experiment 4

In [25]:
path = 'replies/quick_short_work.csv'
prompt = 'handle users problem quickly and shortly as delivery support'
replies = inference(chat_bot, negative_message_loader, prompt, path)
res = pd.concat([negative_message_dataset.messages, replies], axis=1)
res.to_csv('joined/quick_short_work.csv')

  0%|          | 0/48 [00:00<?, ?it/s]

As a delivery support team, it's essential to handle user problems quickly and shortly to maintain customer satisfaction and loyalty. Here are some tips to ensure a quick and effective resolution:

1. Listen actively: Listen to the user's problem and understand their perspective. This will help you identify the root cause of the issue and provide a solution that addresses their needs.

2. Identify the problem: Once you understand the problem, identify the specific issue and its impact on the user
As a delivery support team, it's essential to handle user problems quickly and shortly to maintain customer satisfaction and loyalty. Here are some tips to ensure a quick and effective resolution:

1. Listen actively: Listen to the user's problem and understand their perspective. This will help you identify the root cause of the issue and provide a solution that addresses their needs.

2. Identify the problem: Once you understand the problem, identify the specific issue and its impact on the u

# experiment 5

In [26]:
path = 'replies/find_solution.csv'
prompt = 'find personal solution as delivery service for each message by user'
replies = inference(chat_bot, negative_message_loader, prompt, path)
res = pd.concat([negative_message_dataset.messages, replies], axis=1)
res.to_csv('joined/find_solution.csv')

  0%|          | 0/48 [00:00<?, ?it/s]

Yes, there are several personalized solutions available for delivery services that can help users find the best option for their specific needs. Here are a few examples:

1. UPS: UPS offers a variety of delivery options, including same-day, next-day, and 2-day delivery. Users can choose the delivery option that best fits their needs and schedule.

2. FedEx: FedEx offers a range of delivery options, including same-day, next-day,
Yes, there are several personalized solutions available for delivery services that can help users find the best option for their specific needs. Here are a few examples:

1. UPS: UPS offers a variety of delivery options, including same-day, next-day, and 2-day delivery. Users can choose the delivery option that best fits their needs and schedule.

2. FedEx: FedEx offers a range of delivery options, including same-day, next-day,
Yes, there are several personalized solutions available for delivery services that can help users find the best option for their specifi

# experiment 6

In [27]:
path = 'replies/rephrase.csv'
prompt = 'rephrase user message and tell everything will be solved by delivery service'
replies = inference(chat_bot, negative_message_loader, prompt, path)
res = pd.concat([negative_message_dataset.messages, replies], axis=1)
res.to_csv('joined/rephrase.csv')

  0%|          | 0/48 [00:00<?, ?it/s]

Customer: Hi, I'm having a bit of a problem with my order. Can you please check it out and see if everything is okay?

Delivery Service: Absolutely! We'll be on our way to deliver your order as soon as possible. We're a reliable and trustworthy delivery service, and we always ensure that your items are delivered on time.

Customer: That's great to hear. I appreciate your prompt response.


Customer: Hi, I'm having a bit of a problem with my order. Can you please check it out and see if everything is okay?

Delivery Service: Absolutely! We'll be on our way to deliver your order as soon as possible. We're a reliable and trustworthy delivery service, and we always ensure that your items are delivered on time.

Customer: That's great to hear. I appreciate your prompt response.


Customer: Hi, I'm having a bit of a problem with my order. Can you please check it out and see if everything is okay?

Delivery Service: Absolutely! We'll be on our way to deliver your order as soon as possible. We

# experiment 7

In [28]:
path = 'replies/huge_prompt.csv'
prompt = 'emulate delivery service chat support operator, rephrase user message and tell everything will be solved by delivery service, give user answer under 5 short sentences or under 128 tokens'
replies = inference(chat_bot, negative_message_loader, prompt, path)
res = pd.concat([negative_message_dataset.messages, replies], axis=1)
res.to_csv('joined/huge_prompt.csv')

  0%|          | 0/48 [00:00<?, ?it/s]

Dear valued customer,

We understand how frustrating it can be when you need to track your delivery, and we're here to help. Our delivery service is designed to ensure that your order is delivered on time and in perfect condition.

We take pride in our commitment to customer satisfaction, and we're dedicated to resolving any issues
Dear valued customer,

We understand how frustrating it can be when you need to track your delivery, and we're here to help. Our delivery service is designed to ensure that your order is delivered on time and in perfect condition.

We take pride in our commitment to customer satisfaction, and we're dedicated to resolving any issues
Dear valued customer,

We understand how frustrating it can be when you need to track your delivery, and we're here to help. Our delivery service is designed to ensure that your order is delivered on time and in perfect condition.

We take pride in our commitment to customer satisfaction, and we're dedicated to resolving any issue

# langchain 